# Use after get_artists_url.ipynb

## Fills the artists dataset with information from mutualart.com

#### initially inpath (artists.csv) contains names and corresponding links to mutalart pages.

In [1]:
import sys
try:
    sys.path.insert(0, "/usr/lib/python3.7/site-packages")
except FileNotFoundError:
    pass

import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

In [4]:
inpath = "artists.csv"
df = pd.read_csv(inpath)
del(df["Unnamed: 0"])
#df.head(10)

#### Parce information for columns: 
#### Country
#### Born
#### Died
#### Info (additional information)

In [5]:
def get_additional(session, df):
    arr = []
    for i in range(df.shape[0]):
        print("\r         ", i, end="")
        tmp = [""]*4
        URL   = df.iloc[i][1]
        if(str(URL) == "nan"):
            arr += [tmp]
            continue
        agent = UserAgent()
        agent.chrome
        page  = s.get(URL, headers={'User-Agent': agent.chrome})
        text  = BeautifulSoup(page.text, "html.parser")
        essence = text.find_all(attrs={"class": "desc col-xs-12"})
        if(len(essence) != 0):
            essence = essence[0].get_text().replace(",", "").replace("\n", "").replace("\t", "")
        else:
            arr += [tmp]
            continue
        essence = essence.split("|")
        tmp[0] = essence[0].strip().rstrip()
        if(len(essence) <= 1):
            arr += [tmp]
            continue
        essence = essence[1].split("-")
        if(len(essence) <= 1):
            arr += [["", "", "", ""]]
            continue
        tmp[1], tmp[2] = essence[0].strip().rstrip(), essence[1].strip().rstrip()
        biginf = [text.find_all(attrs={"class": "bio", "itemprop": "description"})[0].get_text()]
        tmp[3] = biginf
        arr += [tmp]
    return arr

In [6]:
s = requests.Session()
arr = get_additional(s, df)

          160

In [12]:
df["Country"] = [x[0] for x in arr]
df["Born"] = [x[1] for x in arr]
df["Died"] = [x[2] for x in arr]
df["Info"] = [x[3] for x in arr]

In [16]:
df.iloc[110:120]

,Artists,URL,Country,Born,Died,Info
110,Andrea Mantegna,https://www.mutualart.com/Artist/Andrea-Manteg...,Italian,1431,1506,"[""\r\n Andrea M..."
111,Jeff Koons,https://www.mutualart.com/Artist/Jeff-Koons/AB...,NaN,NaN,NaN,NaN
112,Gino Severini,https://www.mutualart.com/Artist/Gino-Severini...,Italian,1883,1966,[\r\n Gino Seve...
113,Max Beckmann,https://www.mutualart.com/Artist/Max-Beckmann/...,German,1884,1950,[\r\n Max Beckm...
114,Georgia O′Keeffe,NaN,,,,
115,Cy Twombly,https://www.mutualart.com/Artist/Cy-Twombly/C5...,American,1928,2011,[\r\n Edwin Par...
116,Lucio Fontana,https://www.mutualart.com/Artist/Lucio-Fontana...,Italian,1899,1968,[\r\n Influenti...
117,Paul Gauguin,https://www.mutualart.com/Artist/Paul-Gauguin/...,French,1848,1903,[\r\n French ar...
118,Marc Chagall,https://www.mutualart.com/Artist/Marc-Chagall/...,Russian,1887,1985,[\r\n Marc Chag...
119,Tang Yin,https://www.mutualart.com/Artist/Tang-Yingwei/...,,,,


In [15]:
df.to_csv(inpath)